# Imports

In [114]:
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np
import pandas as pd

# Set-Up Labels

In [115]:
labels = pd.read_csv('converted_keras/labels.txt', header = None)

labels = labels.rename(columns = {0:'Names'})
new_index = []
new_labels = []

for x in labels['Names']:
    new_index.append(int(x.split(" ", 1)[0]))  # to ensure it is not appended as a list
    new_labels.append(x.split(" ", 1)[1])
    
prediction_dict = dict(zip(new_index, new_labels))

# Code copied from Google 

In [116]:
# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model (set compile to False)
model = tensorflow.keras.models.load_model('converted_keras/keras_model.h5', compile = False)


In [117]:
# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1.
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open('img/test/radcliffe_0.jpg')

#resize the image to a 224x224 with the same strategy as in TM2:
#resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.ANTIALIAS)

#turn the image into a numpy array
image_array = np.asarray(image)

# display the resized image
# image.show()

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

# Load the image into the array
data[0] = normalized_image_array

# run the inference
prediction = model.predict(data)
print(prediction)
print(prediction_dict[2])

[[0.27087224 0.02453597 0.7045918 ]]
Daniel Radcliffe


In [ ]:
# This allows it to 